厳密対角化のコード

In [1]:
module ExactDiag
using LinearAlgebra
using PrettyTables
using Printf
using LaTeXStrings, IJulia
using FunctionWrappers
import FunctionWrappers: FunctionWrapper

#キャッシュ変数
const _dim = Ref(1)
const _site = Ref(1)
const _states = Ref(Vector{Vector{Int}}())


#エクスポート
export init, State, BraState, SpinOp, bra, ket, braket, opket, braop, braopket, SpinOp, Basis, Op, sum_j, s, eigen_state, matrix, u1_sort, f_op, flip, shift, _dim, _site, _states, show_H, id, eigenvalue, z2_sort, rs, z2_eigen, block_diag, sym_sort, show_origin

#1サイトの次元dimとサイト数siteの初期設定
function init(dim::Int, site::Int)
  _dim[] = dim
  _site[] = site
  _states[] = [Nary_trans(i - 1) for i in 1:dim^site]
end

#進数変換
function Nary_trans(t::Int)
  n = Vector{Int}(undef, _site[])
  for i in 1:_site[]
    n[_site[]+1-i] = t % _dim[]
    t = div(t, _dim[])
  end
  n
end

#進数逆変換
function Nary_reverse(n::Vector{Int})
  t = 1
  for i in 1:_site[]
    if n[_site[]+1-i] < 0 || n[_site[]+1-i] >= _dim[]
      return 0
    end
    t += n[_site[]+1-i] * _dim[]^(i - 1)
  end
  t
end

#状態ベクトル
struct State
  coeff::Vector{ComplexF64}
  function State(coeff::Vector{ComplexF64})
    if length(coeff) != _dim[]^_site[]
      throw(ArgumentError("The length of coeff must be equal to dim^site."))
    end
    new(coeff)
  end
end

#基底ベクトルの生成
function Basis(i::Int)
  if i < 1 || i > _dim[]^_site[]
    throw(ArgumentError("Index out of bounds."))
  end
  coeff = zeros(ComplexF64, _dim[]^_site[])
  coeff[i] = 1.0 + 0.0im
  State(coeff)
end

#ブラの型定義
struct BraState
  coeff::Vector{ComplexF64}
  function BraState(coeff::Vector{ComplexF64})
    if length(coeff) != _dim[]^_site[]
      throw(ArgumentError("The length of coeff must be equal to dim^site."))
    end
    new(coeff)
  end
end

#ブラへの変換
function bra(a::State)
  coeff = zeros(ComplexF64, _dim[]^_site[])
  for i in 1:length(a.coeff)
    coeff[i] = conj(a.coeff[i])
  end
  BraState(coeff)
end
function bra(a::BraState)
  BraState(copy(a.coeff))
end

#ケットへの変換
function ket(a::BraState)
  coeff = zeros(ComplexF64, _dim[]^_site[])
  for i in 1:length(a.coeff)
    coeff[i] = conj(a.coeff[i])
  end
  State(coeff)
end
function ket(a::State)
  State(copy(a.coeff))
end

#ブラケットの演算
function braket(state1::Union{State,BraState}, state2::Union{State,BraState})
  a = bra(state1)
  b = ket(state2)
  if length(a.coeff) != length(b.coeff)
    throw(ArgumentError("The lengths of the states must be equal."))
  end
  sum = 0.0 + 0.0im
  for i in 1:length(a.coeff)
    sum += a.coeff[i] * b.coeff[i]
  end
  sum
end

#スピン演算子
struct SpinOp
  coeff::ComplexF64
  f::FunctionWrapper{State,Tuple{State}}
end

#一般の演算子
struct Op
  op::Vector{Vector{SpinOp}}
  function Op(op::Vector{Vector{SpinOp}})
    new(op)
  end
  function Op(op1::SpinOp)
    Op([[op1]])
  end
end

#関数のOpへの変換
function f_op(f::FunctionWrapper{State,Tuple{State}})
  Op(SpinOp(1.0, f))
end
function f_op(f::Function)
  fw = FunctionWrapper{State,Tuple{State}}(f)
  return f_op(fw)
end
#単位元の演算子
function id()
  f = FunctionWrapper{State,Tuple{State}}(x -> x)
  return f_op(f)
end
#スピン演算子のOpへの変換
function s(site::Int, op::Char)
  if op != 'z' && op != '+' && op != '-'
    throw(ArgumentError("sInvalid operator. Use 'z', '+', or '-'."))
  end
  states = deepcopy(_states[])
  if op == 'z'
    function z(a::State)
      coeff = copy(a.coeff)
      for i in 1:length(a.coeff)
        coeff[i] *= (-1)^states[i][(site-1)%_site[]+1] / 2
      end
      State(coeff)
    end
    return f_op(z)
  end
  if op == '+'
    function plus(a::State)
      coeff = zeros(ComplexF64, _dim[]^_site[])
      for i in 1:length(a.coeff)
        t = copy(states[i])
        t[(site-1)%_site[]+1] -= 1
        j = Nary_reverse(t)
        if j == 0
          continue
        end
        coeff[j] += a.coeff[i]
      end
      return State(coeff)
    end
    return f_op(plus)
  end
  if op == '-'
    function minus(a::State)
      coeff = zeros(ComplexF64, _dim[]^_site[])
      for i in 1:length(a.coeff)
        t = copy(states[i])
        t[(site-1)%_site[]+1] += 1
        j = Nary_reverse(t)
        if j == 0
          continue
        end
        coeff[j] += a.coeff[i]
      end
      return State(coeff)
    end
    return f_op(minus)
  end
end
#サイト反転
function flip(a::State)
  coeff = zeros(ComplexF64, _dim[]^_site[])
  states = deepcopy(_states[])
  for i in 1:length(a.coeff)
    t = copy(states[i])
    t1 = zeros(Int, _site[])
    for j in 1:_site[]
      t1[j] = t[_site[]+1-j]
    end
    j = Nary_reverse(t1)
    if j == 0
      continue
    end
    coeff[j] += a.coeff[i]
  end
  return State(coeff)
end
flip() = f_op(flip)
#サイト並進
function shift(k::Int=1)
  function shift1(a::State)
    coeff = zeros(ComplexF64, _dim[]^_site[])
    states = deepcopy(_states[])
    for i in 1:length(a.coeff)
      t = copy(states[i])
      t1 = zeros(Int, _site[])
      for j in 1:_site[]
        t1[j] += t[(j-k-1+_site[])%_site[]+1]
      end
      j = Nary_reverse(t1)
      if j == 0
        continue
      end
      coeff[j] += a.coeff[i]
    end
    State(coeff)
  end
  f_op(shift1)
end
#ここを最後にして演算子を加えていく。演算子に引数を取りたい場合には内部で引数を反映したStateからStateへの関数を定義してからその関数をf_opに渡す。引数を取らない場合には単純に関数を定義してから同じ関数名でf_opに渡す。exportも忘れないように。

function Op(op1::Op)
  op1
end

#オーバーロード
import Base: *, +, -, show, ==, isequal, hash
#状態の等価性(Set{State}を定義するため)
function ==(state1::State, state2::State)
  state1.coeff == state2.coeff
end
function ==(state1::BraState, state2::BraState)
  state1.coeff == state2.coeff
end
isequal(x::State, y::State) = x == y
function hash(s::State, h::UInt)
  return hash(s.coeff, h)
end
#演算子の足し算
function +(op1::Op...)
  k = Vector{Vector{SpinOp}}()
  for op2 in op1
    for i in op2.op
      push!(k, i)
    end
  end
  Op(k)
end
#状態の重ね合わせ
function +(state1::Union{State,BraState}...)
  State(sum(state2.coeff for state2 in state1))
end
#状態のスカラー倍
function *(coeff::Union{ComplexF64,Float64}, state::State)
  State(coeff * state.coeff)
end
function *(coeff::Union{ComplexF64,Float64}, state::BraState)
  BraState(coeff * state.coeff)
end
#帰納的な演算子の掛け算
function *(op1::Op)
  op1
end
function *(op1::Op, op2::Op...)
  op3 = *(op2...)
  k = Vector{Vector{SpinOp}}()
  i = 1
  for op11 in op1.op
    for op31 in op3.op
      push!(k, Vector{SpinOp}())
      for op12 in op11
        push!(k[i], op12)
      end
      for op32 in op31
        push!(k[i], op32)
      end
      i += 1
    end
  end
  Op(k)
end
#演算子のスカラー倍
function *(coeff::Union{ComplexF64,Float64}, op1::Op...)
  op2 = *(op1...)
  op3 = Vector{Vector{SpinOp}}()
  for op21 in op2.op
    push!(op3, Vector{SpinOp}())
    for i in 1:length(op21)
      push!(op3[end], SpinOp(i == 1 ? coeff * op21[i].coeff : op21[i].coeff, op21[i].f))
    end
  end
  Op(op3)
end
#演算子の引き算
function -(op1::Op)
  -1.0 * op1
end
function -(op1::Op, op2::Op)
  op1 + (-1.0 * op2)
end
function -(op1::Op, op2::Op...)
  op3 = +(op2...)
  op1 - op3
end
#ブラとケットの引き算
function -(a::State, b::State)
  State(a.coeff - b.coeff)
end
function -(a::BraState, b::BraState)
  BraState(a.coeff - b.coeff)
end
#ブラとケットの演算
function *(a::Union{State,BraState}, b::Union{State,BraState})
  braket(a, b)
end

#演算子とケットの演算
function opket(a::SpinOp, b::State)
  State(a.coeff * a.f(b).coeff)
end
#複数の演算子とケットの演算
function opket(args...)
  @assert length(args) ≥ 1
  b = args[end]::State
  ops = args[1:end-1]
  b1 = b
  for op in reverse(ops)
    b1 = opket(op::SpinOp, b1)
  end
  b1
end
function opket(b::Op, a::Union{State,BraState})
  a1 = State(zeros(ComplexF64, _dim[]^_site[]))
  a2 = ket(a)
  for op1 in b.op
    a1 += opket(op1..., a2)
  end
  a1
end
function *(b::Op, a::Union{State,BraState})
  opket(b, a)
end

#ブラと演算子の演算(行列形式にしてエルミート共役もいいが、もう少しまともな方法はないものか、一旦凍結)
function braop(state1::Union{State,BraState}, a::SpinOp)
  b = ket(state1).coeff
  mat = matrix(a)
  bra(adjoint(mat) * b)
end

#複数の演算子とブラの演算
function braop(b::Union{BraState,State}, a::SpinOp...)
  b1 = bra(b)
  for op1 in a
    b1 = braop(b1, op1)
  end
  b1
end
function braop(b::Union{BraState,State}, a::Op)
  b1 = bra(b)
  b2 = BraState(zeros(ComplexF64, _dim[]^_site[]))
  for op1 in a.op
    b2 += braop(b1, op1...)
  end
  b2
end
function *(b::Union{BraState,State}, a::Op)
  braop(b, a)
end

#演算子をブラケットで挟む
function braopket(args...)
  @assert length(args) ≥ 2
  a = args[1]::Union{State,BraState}
  ops = args[2:end]
  b = args[end]::Union{State,BraState}
  braket(bra(a), opket(ops...))
end
function braopket(a::Union{State,BraState}, b::Op, c::Union{State,BraState})
  braket(bra(a), opket(b, ket(c)))
end#opを任意の個数にできていないので、Basis(1)*s(1,'z')*s(2,'z')*Basis(1)のように書いてもエラーを吐くので、Basis(1)*(s(1,'z')*s(2,'z'))*Basis(1)としなければならない

#引数の演算子をすべて足し合わせる
function sum_j(mats::Op...)
  ans = mats[1]
  @inbounds for mat in mats[2:end]
    ans += mat
  end
  ans
end# サイト数以外の任意のiで回す場合にはsum_j(Tuple(f(i) for i in 1:4)...)のように書いて内包表記

#\sum_jに従って行列の関数を足し合わせる
function sum_j(f::Function)
  sum_j(Tuple(f(i) for i in 1:_site[])...)
end# サイト数で回す場合にはsum_j(i->f(i))のように書く
#行列表示のフォーマット
function complex_formatter(; digits::Int=1)
  return (v, i, j) -> begin
    if v == 0 + 0im
      @sprintf("%.*f", digits, 0.0)
    elseif isa(v, Complex)
      rea = round(real(v), digits=digits)
      image = round(imag(v), digits=digits)
      if image == 0
        @sprintf("%.*f", digits, rea)
      elseif rea == 0
        @sprintf("%.*fim", digits, image)
      else
        sign = image > 0 ? "+" : "-"
        @sprintf("%.*f%s%.*fim", digits, rea, sign, digits, abs(image))
      end
    elseif isa(v, Number)
      @sprintf("%.*f", digits, v)
    else
      string(v)
    end
  end
end
#vectorの文字列化
function str_vec(v::Vector{Int})
  s = ""
  for i in v
    s *= string(i)
  end
  s
end
#ハミルトニアンの表示
function show_H(A::Matrix{ComplexF64}, digit::Int=1)
  states = deepcopy(_states[])
  pretty_table(A, header=([str_vec(states[i]) for i in 1:size(A, 2)]), row_labels=([str_vec(states[i]) for i in 1:size(A, 2)]), formatters=complex_formatter(digits=digit))
end
#対角化行列の表示
function show_u(A::Matrix{ComplexF64}, Energy::Union{Vector{ComplexF64},Vector{Float64}}, digit::Int=1)
  states = deepcopy(_states[])
  A_t = transpose(A)
  pretty_table(A_t, header=([str_vec(states[i]) for i in 1:size(A, 2)]), row_labels=([complex_to_string(com) for com in Energy]), row_label_column_title="E", formatters=complex_formatter(digits=digit))
end
#行列の生成
function matrix(op1::Op)
  A = Matrix{ComplexF64}(undef, _dim[]^_site[], _dim[]^_site[])
  for i in 1:_dim[]^_site[]
    for j in 1:_dim[]^_site[]
      A[i, j] = braopket(Basis(i), op1, Basis(j))
    end
  end
  A
end
function show(op1::Op, digit1::Int=1, digit2::Int=1)
  A = matrix(op1)
  println("Matrix:")
  show_H(A, digit1)
  e, u = eigen(A)
  println("Eigen:")
  show_u(u, e, digit2)
end
#対角化行列のState化
function eigen_state(op1::Op)
  mat = matrix(op1)
  e, u = eigen(mat)
  coeff = Vector{State}()
  state_tot = _dim[]^_site[]
  for i in 1:state_tot
    push!(coeff, State(u[:, i]))
  end
  coeff
end
#複素数の表示
function complex_to_string(z::ComplexF64, tol::Float64=0.0)
  # 0 判定にトレランスを導入
  re = abs(real(z)) ≤ tol ? 0.0 : real(z)
  im = abs(imag(z)) ≤ tol ? 0.0 : imag(z)

  # ① 0
  if re == 0.0 && im == 0.0
    return "0"

    # ② 純虚数
  elseif re == 0.0
    coef = im == 1.0 ? "" :
           im == -1.0 ? "-" :
           string(im)
    return coef * "i"

    # ③ 純実数
  elseif im == 0.0
    return string(re)

    # ④ 一般複素数
  else
    sign = im > 0 ? "+" : "-"          # 正負で符号を決定
    coeff = abs(im) == 1.0 ? "" : string(abs(im))
    return string(re, sign, coeff, "i")
  end
end
function complex_to_string(x::Float64)
  x
end
#状態の表示
function show(a::State, str1::String="")
  states = _states[]
  str = raw""
  str *= str1
  for i in 1:length(a.coeff)
    if a.coeff[i] == 1.0
      if str == str1
        str *= raw"|" * str_vec(states[i]) * raw"\rangle"
      else
        str *= raw"+" * raw"|" * str_vec(states[i]) * raw"\rangle"
      end
    elseif a.coeff[i] == -1.0
      if str == str1
        str *= raw"|" * str_vec(states[i]) * raw"\rangle"
      else
        str *= raw"+" * raw"|" * str_vec(states[i]) * raw"\rangle"
      end
    elseif imag(a.coeff[i]) == 0 && real(a.coeff[i]) > 0
      if str == str1
        str *= complex_to_string(a.coeff[i]) * raw"|" * str_vec(states[i]) * raw"\rangle"
      else
        str *= raw"+" * complex_to_string(a.coeff[i]) * raw"|" * str_vec(states[i]) * raw"\rangle"
      end
    elseif imag(a.coeff[i]) == 0 && real(a.coeff[i]) < 0
      str *= complex_to_string(a.coeff[i]) * raw"|" * str_vec(states[i]) * raw"\rangle"
    elseif imag(a.coeff[i]) > 0 && real(a.coeff[i]) == 0
      if str == str1
        str *= complex_to_string(a.coeff[i]) * raw"|" * str_vec(states[i]) * raw"\rangle"
      else
        str *= raw"+" * complex_to_string(a.coeff[i]) * raw"|" * str_vec(states[i]) * raw"\rangle"
      end
    elseif imag(a.coeff[i]) < 0 && real(a.coeff[i]) == 0
      str *= complex_to_string(a.coeff[i]) * raw"|" * str_vec(states[i]) * raw"\rangle"
    elseif a.coeff[i] != 0
      str *= raw"+"*raw"(" * complex_to_string(a.coeff[i]) * raw")" * raw"|" * str_vec(states[i]) * raw"\rangle"
    end
  end
  if str == str1
    str *= raw"0"
    display("text/latex", latexstring(str))
  else
    display("text/latex", latexstring(rstrip(str, '+')))
  end
end
#固有値を出そう
function eigenvalue(op1::Op, state1::State)
  if sum(abs.(state1.coeff)) == 0
    throw(ArgumentError("The state must not be zero."))
  end
  state2::State = op1 * state1
  ans = 0.0 + 0.0im
  for i in 1:length(state1.coeff)
    if state1.coeff[i] != 0
      ans = state2.coeff[i] / state1.coeff[i]
      break
    end
  end
  for i in 1:length(state1.coeff)
    if state1.coeff[i] * ans != state2.coeff[i]
      throw(ArgumentError("The state is not an eigenstate of the operator."))
    end
  end
  ans
end
#固有値を出そう
function eigenvalue(state1::State,state2::State)
  if sum(abs.(state1.coeff)) == 0
    throw(ArgumentError("The state must not be zero."))
  end
  ans = 0.0 + 0.0im
  for i in 1:length(state1.coeff)
    if state1.coeff[i] != 0
      ans = state2.coeff[i] / state1.coeff[i]
      break
    end
  end
  for i in 1:length(state1.coeff)
    if state1.coeff[i] * ans != state2.coeff[i]
      throw(ArgumentError("The state must not be zero."))
    end
  end
  ans
end
#固有状態になっていますかね
function eigen_check(state1::State,state2::State)
  if sum(abs.(state1.coeff)) == 0
    throw(ArgumentError("The state must not be zero."))
  end
  ans = 0.0 + 0.0im
  for i in 1:length(state1.coeff)
    if state1.coeff[i] != 0
      ans = state2.coeff[i] / state1.coeff[i]
      break
    end
  end
  for i in 1:length(state1.coeff)
    if state1.coeff[i] * ans != state2.coeff[i]
      return false
    end
  end
  true
end
#U(1)対称性による状態の並び替え
function u1_sort(op1::Op)
  states = Dict{Union{ComplexF64,Float64},Vector{Tuple{Int,State}}}()
  dim_tot = _dim[]^_site[]
  for i in 1:dim_tot
    states[eigenvalue(op1, Basis(i))] = Vector{Tuple{Int,State}}()
  end
  for i in 1:dim_tot
    push!(states[eigenvalue(op1, Basis(i))], (i,Basis(i)))
  end
  states
end#すべてのS^z_iと交換可能な演算子を使いましょう
#n乗根の計算
function nthroots(z::ComplexF64, n::Int)
    if n==2&& imag(z)==0.0
        return [sqrt(real(z)) + 0.0im, -sqrt(real(z)) + 0.0im]
    end
    r = abs(z)
    θ = angle(z)
    roots = Vector{ComplexF64}(undef, n)
    for k in 0:n-1
        roots[k+1] = r^(1/n) * cis((θ + 2π*k)/n)
    end
    return roots
end
#\hat{A}^n=(対角演算子)(nはサイト数以下)を満たす演算子Aについての固有状態
function sym_sort(op1::Op,bound::Int=_site[])
  dim_tot = _dim[]^_site[]
  site= _site[]
  states=Dict{Union{ComplexF64,Float64},Vector{Tuple{Int,State}}}()
  basis_set= Set{Int}()
  for i in 1:dim_tot
    if i in basis_set
      continue
    end
    state1= Basis(i)
    states1 = Vector{State}()
    basises::Int=dim_tot
    eigenvalue1 = 0.0+ 0.0im
    push!(states1, state1)
    basises=min(basises, i)
    for j in 1:bound
      if !eigen_check(Basis(i), opket(op1, state1))
        if j== bound
          throw(ArgumentError("The operator does not satisfy the condition for symmetry sorting."))
        end
        for k in 1:dim_tot
          if state1.coeff[k] != 0.0+0.0im
            basises=min(basises, k)
            push!(basis_set, k)
          end
        end
        state1=opket(op1, state1)
        push!(states1, state1)
      else
        for k in 1:dim_tot
          if state1.coeff[k] != 0.0+0.0im
            basises=min(basises, k)
            push!(basis_set, k)
          end
        end
        eigenvalue1 = eigenvalue(Basis(i), opket(op1, state1))
        break
      end
    end
    states_len = length(states1)
    roots= nthroots(eigenvalue1,states_len)
    for k in 1:states_len
      state2=State(zeros(ComplexF64, dim_tot))
      for l in 1:states_len
        state2+=(roots[k]^(l-1))*states1[l]
      end
      if !haskey(states, roots[k])
        states[roots[k]]=Vector{Tuple{Int,State}}()
      end
      state2=(1/sqrt(sum(([abs(state2.coeff[i])^2 for i in 1:dim_tot]))))* state2
      push!(states[roots[k]],(basises,state2))
    end
  end
  states
end
#書き換えた固有状態でのハミルトニアンの行列
function matrix(op1::Op, ham::Op)
  eigens = sym_sort(op1)
  sorted_keys = sort(collect(keys(eigens)), by=real)
  eigens1 = [eigen[2] for key in sorted_keys for eigen in eigens[key]]
  dim_tot = _dim[]^_site[]
  A = [braopket(eigens1[i],ham,eigens1[j]) for i in 1:dim_tot, j in 1:dim_tot]
  return (eigens,A)
end
#その行列による表示
function show(eigens::Dict{Union{ComplexF64,Float64},Vector{Tuple{Int,State}}}, A::Matrix{ComplexF64}, digit::Int=1)
  sorted_keys = sort(collect(keys(eigens)), by=real)
  eigens1 = [eigen[2] for key in sorted_keys for eigen in eigens[key]]
  states = _states[]
  dim_tot = _dim[]^_site[]
  pretty_table(A, header=([str_vec(states[eigen[1]])*","*complex_to_string(key) for key in sorted_keys for eigen in eigens[key]]), row_labels=([str_vec(states[eigen[1]])*","*complex_to_string(key) for key in sorted_keys for eigen in eigens[key]]), formatters=complex_formatter(digits=digit))
end
#書き換えた固有状態でハミルトニアンを表示
function show(op1::Op, ham::Op,digit::Int=1)
  eigens = sym_sort(op1)
  sorted_keys = sort(collect(keys(eigens)), by=real)
  eigens1 = [eigen[2] for key in sorted_keys for eigen in eigens[key]]
  states = _states[]
  dim_tot = _dim[]^_site[]
  A = [braopket(eigens1[i],ham,eigens1[j]) for i in 1:dim_tot, j in 1:dim_tot]
  pretty_table(A, header=([str_vec(states[eigen[1]])*","*complex_to_string(key) for key in sorted_keys for eigen in eigens[key]]), row_labels=([str_vec(states[eigen[1]])*","*complex_to_string(key) for key in sorted_keys for eigen in eigens[key]]), formatters=complex_formatter(digits=digit))
end
#ブロック対角化しましょう
function block_diag(op1::Op, ham::Op)
  eigens= sym_sort(op1)
  sorted_keys = sort(collect(keys(eigens)), by=real)
  dim_tot = _dim[]^_site[]
  states= _states[]
  H= zeros(ComplexF64, dim_tot, dim_tot)
  u=zeros(ComplexF64, dim_tot, dim_tot)
  e= Vector{Float64}(undef, dim_tot)
  num::Int=1
  for key in sorted_keys
    length_eigens = length(eigens[key])
    A = Matrix{ComplexF64}(undef, length_eigens, length_eigens)
    for i in 1:length_eigens
      for j in 1:length_eigens
        A[i, j] = braopket(eigens[key][i][2], ham, eigens[key][j][2])
      end
    end
    H[num:num+length_eigens-1, num:num+length_eigens-1] = A
    e1,u1 = eigen(A)
    u[num:num+length_eigens-1, num:num+length_eigens-1] = u1
    e[num:num+length_eigens-1] = real.(e1)
    num+= length_eigens
  end
  return (eigens,e,u)
end
#対角化したものの表示
function show(eigens::Dict{Union{ComplexF64,Float64},Vector{Tuple{Int,State}}}, e::Vector{Float64}, u::Matrix{ComplexF64}, digit1::Int=1, digit2::Int=1)
  sorted_keys = sort(collect(keys(eigens)), by=real)
  states = _states[]
  u_t = transpose(u)
  pretty_table(u_t, header=([str_vec(states[eigen[1]])*","*complex_to_string(key) for key in sorted_keys for eigen in eigens[key]]), row_label_column_title="E", row_labels=([complex_to_string(com) for com in e]), formatters=complex_formatter(digits=digit2))
end
#元の表示による対角化行列
function show_origin(eigens::Dict{Union{ComplexF64,Float64},Vector{Tuple{Int,State}}}, e::Vector{Float64}, u::Matrix{ComplexF64}, digit::Int=1)
  sorted_keys = sort(collect(keys(eigens)), by=real)
  states = _states[]
  dim_tot = _dim[]^_site[]
  trans=Matrix{ComplexF64}(undef, dim_tot, dim_tot)
  num=0
  for key in sorted_keys
    for (i,state) in eigens[key]
      num+=1
      for j in 1:dim_tot
        trans[num,j]=state.coeff[j]
      end
    end
  end
  u1=transpose(trans)*u
  u_t = transpose(u1)
  pretty_table(u_t, header=([str_vec(states[i]) for i in 1:size(u_t, 2)]), row_label_column_title="E", row_labels=([complex_to_string(com) for com in e]), formatters=complex_formatter(digits=digit))
end
#基底状態が異なる場合の表示
function show(eigens::Dict{Union{ComplexF64,Float64},Vector{Tuple{Int,State}}}, digit::Int=1)
  sorted_keys = sort(collect(keys(eigens)), by=real)
  for key in sorted_keys
    println("Eigenvalue: ", complex_to_string(key))
    for (i,state) in eigens[key]
      show(state, raw"\textrm{State}\space" * "$i:")
    end
  end
end
# #状態の表示
# function show(u1::Dict{Union{ComplexF64,Float64},Vector{Tuple{Int,State}}})
#   sorted= sort(collect(keys(u1)), by=real)
#   for key in sorted
#     println("Eigenvalue: ", complex_to_string(key))
#     for state in u1[key]
#       show(state[2])
#     end
#   end
# end
function show(u1::Vector{Vector{State}})
  for u2 in u1
    for u3 in u2
      show(u3)
    end
  end
end

#z2対称性による状態の並べ替え
function z2_sort(op1::Op)
  states = Vector{Set{State}}()
  for i in 1:_dim[]^_site[]
    push!(states, Set{State}())
    push!(states[i], Basis(i))
    push!(states[i], op1 * Basis(i))
  end
  states
end
function show(z2::Vector{Set{State}})
  for i in 1:length(z2)
    show(Basis(i), raw"\textrm{State}\space" * "$i:")
    for state in z2[i]
      show(state)
    end
  end
end
#代表元状態の取得
function rs(set1::Set{State})
  dim_tot = _dim[]^_site[]
  for i in 1:dim_tot
    if Basis(i) in set1
      return Basis(i)
    end
  end
  throw(ArgumentError("The set does not contain any basis state."))
end
#z2対称性を持つ演算子の固有状態
function z2_eigen(op1::Op, i::Int, ev::Int)
  if i < 1 || i > _dim[]^_site[]
    throw(ArgumentError("Index out of bounds."))
  end
  if ev != 1 && ev != -1
    throw(ArgumentError("Eigenvalue must be 1 or -1."))
  end
  state = rs(z2_sort(op1)[i])
  (sqrt(length(z2_sort(op1)[i])) / 2) * (state + (ComplexF64(ev) * op1) * state)
end
function z2_eigen(op1::Op, ev::Int)
  if ev != 1 && ev != -1
    throw(ArgumentError("Eigenvalue must be 1 or -1."))
  end
  eigen_states = Vector{State}()
  dim_tot = _dim[]^_site[]
  for i in 1:dim_tot
    push!(eigen_states, z2_eigen(op1, i, ev))
  end
  eigen_states
end
#これは正確に0であるような状態を除いた固有状態である
function z2_eigen(op1::Op)
  eigen_states = Dict{Int,Set{State}}()
  eigen_states[1] = Set{State}()
  eigen_states[-1] = Set{State}()
  for i in 1:_dim[]^_site[]
    if sum(abs.(z2_eigen(op1, i, 1).coeff)) != 0
      push!(eigen_states[1], z2_eigen(op1, i, 1))
    end
    if sum(abs.(z2_eigen(op1, i, -1).coeff)) != 0
      push!(eigen_states[-1], z2_eigen(op1, i, -1))
    end
  end
  eigen_states1=Dict{Int,Vector{State}}()
  eigen_states1[1] = Vector{State}()
  eigen_states1[-1] = Vector{State}()
  for key in keys(eigen_states)
    for state in eigen_states[key]
      push!(eigen_states1[key], state)
    end
  end
  eigen_states1
end
end

Main.ExactDiag

In [12]:
using .ExactDiag
init(2,3)
n=sum_j(i->1/2*id()-s(i,'z'))
H = sum_j(j->1/2*(s(j,'+')*s(j+1,'-')+s(j,'-')*s(j+1,'+'))+1.0*s(j,'z')*s(j+1,'z'))
show_origin(block_diag(id(),H)...,3)
show_origin(block_diag(n,H)...,3)
show_origin(block_diag(flip(),H)...,3)
show_origin(block_diag(shift(),H)...,3)
show(H,2,2)

┌───────┬───────┬────────┬────────┬────────┬────────┬────────┬────────┬───────┐
│     E │   000 │    001 │    010 │    011 │    100 │    101 │    110 │   111 │
├───────┼───────┼────────┼────────┼────────┼────────┼────────┼────────┼───────┤
│ -0.75 │ 0.000 │  0.707 │ -0.707 │  0.000 │  0.000 │  0.000 │  0.000 │ 0.000 │
│ -0.75 │ 0.000 │  0.000 │  0.000 │ -0.707 │  0.000 │  0.707 │  0.000 │ 0.000 │
│ -0.75 │ 0.000 │ -0.408 │ -0.408 │  0.000 │  0.816 │  0.000 │  0.000 │ 0.000 │
│ -0.75 │ 0.000 │  0.000 │  0.000 │ -0.408 │  0.000 │ -0.408 │  0.816 │ 0.000 │
│  0.75 │ 0.000 │ -0.577 │ -0.577 │  0.000 │ -0.577 │  0.000 │  0.000 │ 0.000 │
│  0.75 │ 0.000 │  0.000 │  0.000 │ -0.577 │  0.000 │ -0.577 │ -0.577 │ 0.000 │
│  0.75 │ 1.000 │  0.000 │  0.000 │  0.000 │  0.000 │  0.000 │  0.000 │ 0.000 │
│  0.75 │ 0.000 │  0.000 │  0.000 │  0.000 │  0.000 │  0.000 │  0.000 │ 1.000 │
└───────┴───────┴────────┴────────┴────────┴────────┴────────┴────────┴───────┘
┌───────┬───────┬────────┬────────┬─────

In [3]:
using .ExactDiag
init(2,3)
n=sum_j(i->1/2*id()-s(i,'z'))
show(sym_sort(flip()))

Eigenvalue: -1.0


Eigenvalue: 1.0


nサイトの横磁場Ising模型
$$
\hat{H}=\frac{J}{2}\sum_j\left(\hat{S}^z_j\hat{S}^z_{j+1}-\frac{h}{J}(\hat{S}^+_j+\hat{S}^-_j)\right)\\
\frac{h}{J}=1
$$

In [4]:
using .ExactDiag
hj=1.0
n=2
init(2, n)
H = sum_j(j->s(j, 'z')*s(j+1, 'z') - hj*(s(j, '+') + s(j, '-')))
show(H,2,2)

Matrix:
┌────┬───────┬───────┬───────┬───────┐
│    │    00 │    01 │    10 │    11 │
├────┼───────┼───────┼───────┼───────┤
│ 00 │  0.50 │ -1.00 │ -1.00 │  0.00 │
│ 01 │ -1.00 │ -0.50 │  0.00 │ -1.00 │
│ 10 │ -1.00 │  0.00 │ -0.50 │ -1.00 │
│ 11 │  0.00 │ -1.00 │ -1.00 │  0.50 │
└────┴───────┴───────┴───────┴───────┘
Eigen:
┌──────────┬───────┬───────┬───────┬───────┐
│        E │    00 │    01 │    10 │    11 │
├──────────┼───────┼───────┼───────┼───────┤
│ -2.06155 │ -0.44 │ -0.56 │ -0.56 │ -0.44 │
│     -0.5 │  0.00 │ -0.71 │  0.71 │  0.00 │
│      0.5 │ -0.71 │  0.00 │  0.00 │  0.71 │
│  2.06155 │  0.56 │ -0.44 │ -0.44 │  0.56 │
└──────────┴───────┴───────┴───────┴───────┘


nサイトのXXZ模型
$$
\hat{H}=J\sum_j\left[\frac{1}{2}(\hat{S}_j^+ \hat{S}_{j+1}^- +\hat{S}_j^- \hat{S}_{j+1}^+)+\Delta\hat{S}_j^z \hat{S}_{j+1}^z\right]\\
\Delta=1
$$

In [5]:
using .ExactDiag
Δ=1.0
n=3
init(2, n)
H = sum_j(j->1/2*(s(j,'+')*s(j+1,'-')+s(j,'-')*s(j+1,'+'))+Δ*s(j,'z')*s(j+1,'z'))
show(H,2,2)

Matrix:
┌─────┬──────┬───────┬───────┬───────┬───────┬───────┬───────┬──────┐
│     │  000 │   001 │   010 │   011 │   100 │   101 │   110 │  111 │
├─────┼──────┼───────┼───────┼───────┼───────┼───────┼───────┼──────┤
│ 000 │ 0.75 │  0.00 │  0.00 │  0.00 │  0.00 │  0.00 │  0.00 │ 0.00 │
│ 001 │ 0.00 │ -0.25 │  0.50 │  0.00 │  0.50 │  0.00 │  0.00 │ 0.00 │
│ 010 │ 0.00 │  0.50 │ -0.25 │  0.00 │  0.50 │  0.00 │  0.00 │ 0.00 │
│ 011 │ 0.00 │  0.00 │  0.00 │ -0.25 │  0.00 │  0.50 │  0.50 │ 0.00 │
│ 100 │ 0.00 │  0.50 │  0.50 │  0.00 │ -0.25 │  0.00 │  0.00 │ 0.00 │
│ 101 │ 0.00 │  0.00 │  0.00 │  0.50 │  0.00 │ -0.25 │  0.50 │ 0.00 │
│ 110 │ 0.00 │  0.00 │  0.00 │  0.50 │  0.00 │  0.50 │ -0.25 │ 0.00 │
│ 111 │ 0.00 │  0.00 │  0.00 │  0.00 │  0.00 │  0.00 │  0.00 │ 0.75 │
└─────┴──────┴───────┴───────┴───────┴───────┴───────┴───────┴──────┘
Eigen:
┌───────┬──────┬───────┬───────┬───────┬───────┬───────┬───────┬──────┐
│     E │  000 │   001 │   010 │   011 │   100 │   101 │   110 │  111 │
├

nサイトのhard-core bosson系
$$
\hat{H}=-\frac{J}{2}\sum_j(\hat{S}^+_j\hat{S}^-_{j+1}+\hat{S}^-_j\hat{S}^+_{j+1})
$$

In [6]:
using .ExactDiag
n=3
init(2, n)
H=-1.0/2.0*sum_j(j->s(j,'+')*s(j+1,'-')+s(j,'-')*s(j+1,'+'))
show(H,1,3)

Matrix:
┌─────┬─────┬──────┬──────┬──────┬──────┬──────┬──────┬─────┐
│     │ 000 │  001 │  010 │  011 │  100 │  101 │  110 │ 111 │
├─────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼─────┤
│ 000 │ 0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │ 0.0 │
│ 001 │ 0.0 │  0.0 │ -0.5 │  0.0 │ -0.5 │  0.0 │  0.0 │ 0.0 │
│ 010 │ 0.0 │ -0.5 │  0.0 │  0.0 │ -0.5 │  0.0 │  0.0 │ 0.0 │
│ 011 │ 0.0 │  0.0 │  0.0 │  0.0 │  0.0 │ -0.5 │ -0.5 │ 0.0 │
│ 100 │ 0.0 │ -0.5 │ -0.5 │  0.0 │  0.0 │  0.0 │  0.0 │ 0.0 │
│ 101 │ 0.0 │  0.0 │  0.0 │ -0.5 │  0.0 │  0.0 │ -0.5 │ 0.0 │
│ 110 │ 0.0 │  0.0 │  0.0 │ -0.5 │  0.0 │ -0.5 │  0.0 │ 0.0 │
│ 111 │ 0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │  0.0 │ 0.0 │
└─────┴─────┴──────┴──────┴──────┴──────┴──────┴──────┴─────┘
Eigen:
┌──────┬───────┬────────┬────────┬────────┬────────┬────────┬────────┬───────┐
│    E │   000 │    001 │    010 │    011 │    100 │    101 │    110 │   111 │
├──────┼───────┼────────┼────────┼────────┼────────┼────────┼────────┼───────┤
│ -1

$U(1)$対称性

$S_z^\textrm{tot}$の固有値の順序に並び替える

In [7]:
using .ExactDiag
n=2
init(2, n)

show(u1_sort(sum_j(j->1/2*id()-s(j,'z'))))

Eigenvalue: 0


Eigenvalue: 1.0


Eigenvalue: 2.0


反転変換(flip)と並進変換(shift)

In [8]:
using .ExactDiag
init(2, 4)
show(Basis(2))
show(flip()*Basis(2))
show(shift(1)*Basis(2))

In [9]:
using .ExactDiag
init(2,3)
show(rs(z2_sort(flip())[5]))
show(Basis(5))
show(z2_eigen(flip(), 5, -1))
for i in z2_eigen(flip())[1]
  println(eigenvalue(flip(), i))
end
show(z2_eigen(flip())[1][2])

1.0 + 0.0im
1.0 + 0.0im
1.0 + 0.0im
1.0 + 0.0im
1.0 + 0.0im
1.0 + 0.0im


In [10]:
using .ExactDiag
Δ=1.0
n=2
init(2, n)
H = sum_j(j->1/2*(s(1,'+')*s(2,'-')+s(1,'-')*s(2,'+'))+Δ*s(1,'z')*s(2,'z'))
hat_n = sum_j(j->1/2*id()-s(j,'z'))
show(matrix(flip(), H)..., 2)
show(block_diag(flip(), H)...,2,2)

┌─────────┬─────────┬────────┬────────┬────────┐
│         │ 01,-1.0 │ 00,1.0 │ 01,1.0 │ 11,1.0 │
├─────────┼─────────┼────────┼────────┼────────┤
│ 01,-1.0 │   -1.50 │   0.00 │   0.00 │   0.00 │
│  00,1.0 │    0.00 │   0.50 │   0.00 │   0.00 │
│  01,1.0 │    0.00 │   0.00 │   0.50 │   0.00 │
│  11,1.0 │    0.00 │   0.00 │   0.00 │   0.50 │
└─────────┴─────────┴────────┴────────┴────────┘
┌──────┬─────────┬────────┬────────┬────────┐
│    E │ 01,-1.0 │ 00,1.0 │ 01,1.0 │ 11,1.0 │
├──────┼─────────┼────────┼────────┼────────┤
│ -1.5 │    1.00 │   0.00 │   0.00 │   0.00 │
│  0.5 │    0.00 │   0.00 │   1.00 │   0.00 │
│  0.5 │    0.00 │   1.00 │   0.00 │   0.00 │
│  0.5 │    0.00 │   0.00 │   0.00 │   1.00 │
└──────┴─────────┴────────┴────────┴────────┘
